In [207]:
import pandas as pd
import numpy as np
from statistics import mean
import sqlite3
import importlib

import cleaning

%load_ext autoreload
%autoreload 2

importlib.reload(cleaning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<module 'cleaning' from 'd:\\escola\\fac\\4o_ano\\1_semestre\\ac-feup\\jupyters\\cleaning.py'>

In [208]:
test = False

con = sqlite3.connect("../database/banking_data")
cur = con.cursor()

accounts = cleaning.clean_account(con)
cards = cleaning.clean_card(con, test)
clients = cleaning.clean_client(con)
districts = cleaning.clean_district(con)
loans = cleaning.clean_loans(con, test)
trans = cleaning.clean_trans(con, test)
disp = cleaning.pd_disp(con)

## Creation of a new Transactions Dataframe

The new transactions consists on a dataframe where the transactions from the same account are all summarized in 1 row: `min no. trans`, `max no. trans`, `avg no. trans`, `min_balance`, `max_balance` and `avg_balance`.

In [197]:
# Sort values by account ID to make the job easier
trans = trans.sort_values(by="account_id")

aux = 0

# Create new dataframe with the columns needed
trans_final = pd.DataFrame(columns=["account_id", "no. movements", "min no. trans", "max no. trans", "avg no. trans", "min balance", "max balance", "avg balance"])

for i in loans["account_id"]:
    if i != aux:
        trans_test = trans.loc[trans['account_id'] == i]

        # Get the values for each column of the new dataframe
        no_movements = trans_test.shape[0]
        min_no_trans = min(pd.to_numeric(trans_test["amount"]))
        max_no_trans = max(pd.to_numeric(trans_test["amount"]))
        avg_no_trans = mean(pd.to_numeric(trans_test["amount"]))
        min_balance = min(pd.to_numeric(trans_test["balance"]))
        max_balance = max(pd.to_numeric(trans_test["balance"]))
        avg_balance = mean(pd.to_numeric(trans_test["balance"]))
        
        # Append to the new dataframe
        trans_final = trans_final.append({"account_id": i, "no. movements": no_movements, "min no. trans": min_no_trans, "max no. trans": max_no_trans, "avg no. trans": avg_no_trans, "min balance": min_balance, "max balance": max_balance, "avg balance": avg_balance}, ignore_index=True)
        
        aux = i

## Client-District Merge

Merge the clients with the districts using the `district_id` information.

In [198]:
# Add district columns to the right side of the clients columns 
client_district = clients.merge(districts, left_on="district_id", right_on="code")

# Drop irrelevant columns
client_district = client_district.drop(columns=["name", "district_id", "region"])

# Rename necessary columns so we don't have merge conflicts later
client_district = client_district.rename(columns={col:("client_district "+ col) for col in client_district.columns[4:]})

## Account-District Merge

In [199]:
# Add district columns to the right side of the accounts columns 
account_district = accounts.merge(districts, left_on="district_id", right_on="code")

# Drop irrelevant columns
account_district = account_district.drop(columns=["name", "district_id", "code", "region"])

# Rename necessary columns so we don't have merge conflicts later
account_district = account_district.rename(columns={col:("account_district "+ col) for col in account_district.columns[4:]})

## Card-Client Merge

### Useful Renames

In [200]:
# Renamed so it doesn't get any problems while merging with the dispositions
cards = cards.rename(columns={"type":"card_type", "issued":"card_issued"})

### Merging

#### 1. Card-Disposition Merge

In [201]:
# Add disposition columns to the right side of the cards columns 
card_client = cards.merge(disp)

# Drop irrelevant columns
card_client = card_client.drop(columns=["card_id", "disp_id", "client_id", "type"])

# Get Clients that have Loans but don't have a Card
loan_client_no_card = []
for i in loans["account_id"]:
    if not i in list(card_client["account_id"]):
        loan_client_no_card.append(i)

# Get Clients that have a Card but don't have Loans
card_client_no_loan = []
for i in card_client["account_id"]:
    if not i in list(loans["account_id"]):
        card_client_no_loan.append(i)

# Erase account that have cards but don't have a loan
card_client = card_client.drop(index=card_client[card_client["account_id"].isin(card_client_no_loan)].index)

# Add accounts that have loans but don't have a card
loan_client_no_card = np.array([loan_client_no_card, ["None"] * len(loan_client_no_card), [0] * len(loan_client_no_card)]).transpose()
loan_client_no_card = pd.DataFrame(loan_client_no_card, columns=["account_id", "card_type", "card_issued"])

# Add accounts that have loans but don't have a card to the accounts that have loans and have a card
card_client = card_client.append(loan_client_no_card)
card_client["account_id"] = card_client["account_id"].astype(int)

## Account-Client Merge

In [202]:
# Merge the disposition columns with the accounts (with district information) columns
account_client = disp.merge(account_district)

# Merge the previous table with the clients (with the district information) columns
account_client = account_client.merge(client_district, on="client_id")

# The only loans that are relevant are the ones that belong to the owner
account_client = account_client[account_client["type"] == "OWNER"]

# Drop irrelevant columns: client id and type (it's always OWNER)
account_client = account_client.drop(columns=["client_id", "type"])

#client_district.sort_values(by="client_id")
account_client

,disp_id,account_id,frequency,date,no. of inhabitants,account_district no. of municipalities with inhabitants < 499,account_district no. of municipalities with inhabitants 500-1999,account_district no. of municipalities with inhabitants 2000-9999,account_district no. of municipalities with inhabitants >10000,account_district no. of cities,...,client_district ratio of urban inhabitants,client_district average salary,client_district unemploymant rate '95,client_district unemploymant rate '96,client_district unemploymant_growth,client_district no. of enterpreneurs per 1000 inhabitants,client_district no. of commited crimes '95,client_district no. of commited crimes '96,client_district crime_growth,client_district total_crime
0,1,1,monthly issuance,24-03-1995,70699,60,13,2,1,4,...,65.3,8968,2.83,3.35,0.52,131,1740,1910,170,3650
1,2,2,monthly issuance,26-02-1993,1204953,0,0,0,1,1,...,100.0,12541,0.29,0.43,0.14,167,85677,99107,13430,184784
3,4,3,monthly issuance,07-07-1997,95616,65,30,4,1,6,...,51.4,9307,3.85,4.43,0.58,118,2616,3040,424,5656
5,6,4,monthly issuance,21-02-1996,107870,84,29,6,1,6,...,58.0,8754,3.83,4.31,0.48,137,3804,3868,64,7672
6,7,5,monthly issuance,30-05-1997,58796,22,16,7,1,5,...,51.9,9045,3.13,3.60,0.47,124,1845,1879,34,3724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5363,13623,11333,monthly issuance,26-05-1994,112065,95,19,7,1,8,...,69.4,11277,1.25,1.44,0.19,127,5179,4987,-192,10166
5364,13647,11349,weekly issuance,26-05-1995,1204953,0,0,0,1,1,...,100.0,12541,0.29,0.43,0.14,167,85677,99107,13430,184784
5366,13660,11359,monthly issuance,01-10-1994,117897,139,28,5,1,6,...,53.8,8814,4.76,5.74,0.98,107,2112,2059,-53,4171
5367,13663,11362,monthly issuance,14-10-1995,106054,38,25,6,2,6,...,63.1,8110,5.77,6.55,0.78,109,3244,3079,-165,6323


## Final Merge: Loan

### Useful Rename

In [203]:
loans = loans.rename(columns={"date":"loan_date", "amount":"loan_amount", "duration":"loan_duration"})
trans = trans.rename(columns={"amount":"no. transactions"})
account_client = account_client.rename(columns={"frequency":"account_frequency", "date": "account_creation"})


### Merging

In [204]:
# Merge accounts informations into loans columns
loan_final = loans.merge(account_client, on="account_id")

# Merge cards with clients informations into loans columns
loan_final["account_id"] = loan_final["account_id"].astype(int)
loan_final = loan_final.merge(card_client, on="account_id")

# Merge transactions informations into loans colums
trans_final["account_id"] = trans_final["account_id"].astype(int)
loan_final = loan_final.merge(trans_final, on="account_id")

# Drop irrelevant columns
loan_final = loan_final.drop(columns=["account_id", "disp_id"])

# Fill loan_success with None (avoid errors related to DB)



## Add to Database

In [205]:
if test:
    loan_final.to_sql('loan_united_test', con=con, if_exists='replace')
else:
    loan_final.to_sql('loan_united_train', con=con, if_exists='replace')
    
con.close()

D:\outros\Anaconda\lib\site-packages\pandas\core\generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


## Add to CSV File

In [206]:

loan_final = loan_final.rename(columns={'status' : 'loan_success'})
loan_final.to_csv("../csvs/loan_united_"+("test" if test else "train") + ".csv", index=False)
loan_final

,loan_id,loan_date,loan_amount,loan_duration,payments,loan_success,account_frequency,account_creation,no. of inhabitants,account_district no. of municipalities with inhabitants < 499,...,client_district total_crime,card_type,card_issued,no. movements,min no. trans,max no. trans,avg no. trans,min balance,max balance,avg balance
0,5314,05-07-1993,96396,12,8033,-1,weekly issuance,22-03-1993,94812,15,...,5789,None,0,4,1100.0,9900.0,5025.000000,1100.0,20100.0,12250.000000
1,5316,11-07-1993,165960,36,4610,1,monthly issuance,13-02-1993,112709,48,...,5472,None,0,37,2.9,54300.0,11015.635135,700.0,120512.8,52083.859459
2,6863,28-07-1993,127080,60,2118,1,monthly issuance,08-02-1993,77917,85,...,4202,None,0,24,48.6,19065.0,5417.458333,800.0,49590.4,30060.954167
3,5325,03-08-1993,105804,36,2939,1,monthly issuance,30-01-1993,107870,84,...,12899,None,0,25,14.6,26448.0,8253.080000,1000.0,65898.5,41297.480000
4,7240,06-09-1993,274740,60,4579,1,weekly issuance,14-02-1993,1204953,0,...,3022,None,0,27,30.0,63366.0,18945.966667,600.0,122893.1,57188.211111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,6818,12-12-1996,155616,48,3242,1,monthly issuance,21-01-1995,226122,32,...,18880,None,0,172,14.6,36960.0,4895.974419,200.0,75294.1,44197.509884
324,5625,15-12-1996,222180,60,3703,-1,monthly issuance,29-11-1995,45714,52,...,1706,None,0,59,14.6,52600.0,11725.494915,800.0,130659.1,55230.444068
325,6805,21-12-1996,45024,48,938,1,monthly issuance,21-05-1996,285387,0,...,19986,None,0,39,14.6,31636.5,7779.815385,800.0,63659.3,41994.907692
326,7233,25-12-1996,115812,36,3217,1,monthly issuance,20-05-1995,93931,74,...,3787,None,0,124,14.6,50800.0,13280.084677,1100.0,119527.2,56646.516129
